In [1]:
import os
import yaml

os.chdir('Z:/Covid19/Covid19_OMOP/new_pipeline/')
print(os.getcwd())

import pandas as pd
from omop_etl.load import Loader
# from .omop_etl.stage import STAGE

from pydma.utils import find

Z:\Covid19\Covid19_OMOP\new_pipeline


In [2]:
omop = Loader('config.yml')
STAGE = omop.stage

In [3]:
stage_t = omop.list_tables(in_schema=['stage'])
stage_t['Count'] = stage_t.Table.apply(lambda t: omop.row_count(t, schema='stage'))

In [4]:
stage_t_dict = stage_t[['Table','Count']].to_dict()
# stage_t_dict

In [5]:
stage_t_dict['Count'][7]

162226

In [6]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [7]:
loaded = omop.config.load

result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                idx = find_key(stage_t_dict['Table'], stg_name)
                count = stage_t_dict['Count'][idx]
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            idx = find_key(stage_t_dict['Table'], stg_name)
            count = stage_t_dict['Count'][idx]
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Count'])
# table_counts.head(50)

In [8]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['RC_Load'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['RC_DeId'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))
count_diff['RC_Preload'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='preload') if t not in ('death','person','visit_occurrence') else 0)
count_diff

,Table,Count,RC_Load,RC_DeId,RC_Preload
0,condition_occurrence,66999615,56799066,56799066,69171092
1,death,6179,6179,6179,0
2,drug_exposure,73550248,73407336,73407336,73442870
3,measurement,355974353,421425095,421425095,411548640
4,observation,71380058,88073005,88073005,71485504
5,person,204934,204933,204933,0
6,procedure_occurrence,26567539,12008779,12008779,26461938
7,visit_occurrence,23733517,23655952,23655952,0


In [9]:
# t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()